In [ ]:
import numpy as np
import xarray as xr
import pickle
import eddytools as et

In [ ]:
path = "/path/to/model/output/" 
eddypath = "/path/to/tracked/eddies/"

### Eddy detection
Eddy detection is performed on the extended domain in order to detect eddies that span across the periodic boundary of the model domain.

In [ ]:
for y in np.arange(201, 301):
    year = f"{y:04}"
    print("detecting in year", str(year))
    # loading the interpolated data for detection
    data_int = xr.open_mfdataset(eddypath + 'interp_data.extend800.' + str(year) + '*.nc', 
                                 concat_dim="time", combine="nested", data_vars='minimal', 
                                 coords='minimal', compat='override')
    for k in [15]:
        # defining the detection parameters for eddytools
        print("on level", str(k))
        detection_parameters = {'model': 'MITgcm',
                                'grid': 'cartesian',
                                'hemi': 'south',
                                'start_time': str(year) + '-01-01', # time range start
                                'end_time': str(year) + '-12-30', # time range end
                                'calendar': '360_day', # calendar, must be either 360_day or standard
                                'lon1': 0.2e6, # minimum longitude of detection region
                                'lon2': 3.0e6, # maximum longitude
                                'lat1': 0.1e6, # minimum latitude
                                'lat2': 3.1e6, # maximum latitude
                                'res': 10., # resolution of the fields in km
                                'min_dep': 100, # minimum ocean depth where to look for eddies in m
                                'OW_thr': data_int, # 
                                'OW_thr_name': 'OW_std', # Okubo-Weiss threshold for eddy detection
                                'OW_thr_factor': -0.3, # Okubo-Weiss parameter threshold
                                'Npix_min': 20, # minimum number of pixels (grid cells) to be considered as eddy
                                'Npix_max': 500, # maximum number of pixels (grid cells)
                                'no_long': False,
                                'no_two': False
                                } 
        # detect eddies in the extended domain
        eddies_extended = et.detection.detect_OW(data_int.isel(z=k), detection_parameters, 
                                                'OW', 'vort', use_bags=True)
        eddies = {}
        for t in np.arange(0, len(eddies_extended)):
            eddies[t] = {}
            eddy = 0
            # cutout only the "real" domain from the extendend one
            for e in np.arange(0, len(eddies_extended[t])):
                if ((eddies_extended[t][e]['lon'][0] >= 0.4e6) 
                    | (eddies_extended[t][e]['lon'][0] <= 2.8e6)):
                    eddies[t][eddy] = eddies_extended[t][e]
                    if eddies[t][eddy]['lon'][0] >= 2.4e6:
                        eddies[t][eddy]['lon'][0] -= 2.4e6
                        eddies[t][eddy]['eddy_i'] -= 240
                    eddies[t][eddy]['eddy_i'][eddies[t][eddy]['eddy_i'] < 0] += 240
                    eddies[t][eddy]['eddy_i'][eddies[t][eddy]['eddy_i'] >= 240] -= 240
                    eddy += 1
        # write eddies to disk
        for i in np.arange(0, len(eddies)):
            datestring = str(eddies[i][0]['time'])[0:10]
            with open(eddypath + 'eddies/eddies_'
                      + datestring + '.pickle', 'wb') as f:
                pickle.dump(eddies[i], f, pickle.HIGHEST_PROTOCOL)
            f.close()